# Index Aligned Data Comparison Tool
This notebook starts with a demonstration using fake data described in the sibling README file.

In [ ]:
"""
Module Imports and configuration
"""
# Data Modules
import pandas as pd
import numpy as np
import openpyxl
from pprint import pprint

# config
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

## Steps 1 and 2 and 3
1. Load Data - Read data from source, in this case csv files, into 2 or more dataframes
2. Ensure the dataframes share the same column names
3. Add each of the dataframes to a dictionary with an appropriate name

In [ ]:
'''Define a few constants'''
# The name of the index column in your data
indexColName = 'patient_id'
# Name of the columns that will be present in both original dataframes you wish to compare
compareCols = ['cv','coag','hep','nerv','renal','resp']

# define paths to example csv files
goldStandardFilePath = 'exampledata/goldStandard.csv'
computedFilePath = 'exampledata/computed.csv'

# use Pandas to Load the files
goldStandardDf = pd.read_csv(goldStandardFilePath)
computedDf = pd.read_csv(computedFilePath)

# drop rows with null values
goldStandardDf.dropna(inplace=True)
computedDf.dropna(inplace=True)

# set indexes
goldStandardDf.set_index(indexColName, inplace=True)
computedDf.set_index(indexColName, inplace=True)

# Columns - Here we filter both tables down to the columns we wish to compare
# NOTE: Your incoming dataframes must have the columns in compareCols.  If the load process
# loaded them with different names, you will need to add a renaming step prior to this step
goldStandardDf = goldStandardDf.filter(compareCols)
computedDf = computedDf.filter(compareCols)

# final step add your dataframes to a dictionary
# This allows us to keep track of their content with a named key
dfDict = {
    'Gold Standard': goldStandardDf,
    'Computed':computedDf
}

### Review the loaded data
When reviewing the data check for:
1. column names are the same
1. data types are the same

If either of these are different, add a few lines of code in the load process to resolve that now

In [ ]:
# review the gold standard data
print(goldStandardDf.info())
goldStandardDf.head()

In [ ]:
print(computedDf.info())
computedDf.head()

## Step 4 - Index Alignment
Here we make sure that both dataframes, computedDf and goldStandardDf contain the same records by making sure the indexes in all of the dataframes in our dfDict only contain records in all the dataframes.

In [ ]:
def alignIndexes(dfDict):
    '''
    Takes list of N dataframes that are already indexed with the same values
    Returns a list N dataframes containing only the indicies found in the intersection of
    all index values
    '''
    indexSetList = [set(df.index.to_list()) for df in dfDict.values()]
    commonIndexSet = set.intersection(*indexSetList)
    print('There are {} intersecting index values'.format(len(commonIndexSet)))
    dfRetDict = {}
    for k,v in dfDict.items():
        print('Data Source = {}'.format(k))
        print('{} entries before index alignment'.format(len(v)))
        dfAligned = v.loc[list(commonIndexSet)]
        print('{} entries after index alignment'.format(len(dfAligned)))
        dfRetDict[k] = dfAligned
    return dfRetDict

In [ ]:
dfAlignedDict = alignIndexes(dfDict)
pprint(dfAlignedDict)

## Step 5 - Merge the index aligned dataframes together
When the frames are merged, the column names will overlap so pandas will add a 'grouping layer' to the column index that ensures the data from each frame is faithfully represented in the final frame.  The grouping layer names come from the names assigned to each frame in step 3.  This allows us to track the provenance of all data in the merged dataframe.

In [ ]:
# Step 2 - glue dataframes together
# Glues the computed dataframe on to the right edge of the gold standard dataframe
# adds a level to the column index which acts as a grouper
# this denotes which data was from which source
df_all = pd.concat(dfAlignedDict.values(),
                   keys=dfAlignedDict.keys(),
                   axis='columns')
df_all

### Arrange the columns pairwise
Purely for asthetics

In [ ]:
# Step 3 - Interleave columns
# By swapping the column name grouping levels we can arrange each subscores data value with it's sibling source
df_final = df_all.swaplevel(axis='columns')[sorted(set(x[1] for x in df_all.columns))]
df_final

## Step 6 - Compare the Data: Visual Diff

In [ ]:
'''
This function is the primary key to visually grepping the data differences and obtaining a short report
about how many (absolute count/relative proportion) of differences there are
'''
def styleDifferences(df, groupAssumedCorrect=None, color='yellow'):
    '''
    Function that consumes an index aligned dataframe in which each column has a sibling
    from a different 'group'
    This will do pair wise directional comparisons from one sibling to another, highlighting
    discordances with the color of your choosing
    arguments:
    1. indexAlignedDf
    2. groupAssumedCorrect - group assumed to contain ground truth.
    '''
    style = 'background-color: {}'.format(color)
    # sets up a cross section of the dataframe to compare against
    # This carves out the values from the group (computed|goldstandard you wish to treat as ground truth)
    # level = -1 carves out the innermost level of the column grouping levels
    goldStandard = df.xs(groupAssumedCorrect, axis='columns', level=-1)
    # builds a new dataframe with cell highlighting
    # DataFrame constructor is pd.DataFrame(data, index, column_names)
    # conditionally applied when pairwise columnar values are not equal
    # pandas ne() function is tailor made for this operation
    styledDataframe = pd.DataFrame(np.where(
                                        df.ne(goldStandard, axis='columns', level=0),
                                        style,  # apply style when goldStandard cross section != dataframe value
                                        ''), # Don't apply style when the values are equal
                                   index=df.index, 
                                   columns=df.columns)
    return styledDataframe

def computePercentDiscordance(df, groupAssumedCorrect=None):
    '''
    Computes the percent discordance between groud truth group values
    and the alternative group
    '''
    goldStandard = df.xs(groupAssumedCorrect, axis='columns', level=-1)
    return round((df[df.ne(goldStandard, axis='columns', level=0)].count()/df.count())*100,2)

In [ ]:
# Apply the diff 
df_final.style.apply(styleDifferences, axis=None, groupAssumedCorrect='Gold Standard', color='red')

## Step 6 - Compare the Data: Percent Discordance
Rough metric to summarize how much discordance and in which columns for frames that are too large to visualize easily.

In [ ]:
computePercentDiscordance(df_final, groupAssumedCorrect='Gold Standard')

# Common Patterns found in visual diffs
When the discordance is fairly sparse, it can be assumed that the discordances are either because one of the reviewers is wrong, data was not avialable to one of the reviewers, or it was an edged case and judgment or defaults were used instead.

However, when the visual discordances have a pattern to them they get more interesting and usually easier to solve!  Here we present two patterns we see commonly:
1. Systematic error in a metric - columnar disagreement
1. Systematic error in a record - index disagreement

In [ ]:
'''Metric Error'''
'''Define a few constants'''
# The name of the index column in your data
indexColName = 'patient_id'
# Name of the columns that will be present in both original dataframes you wish to compare
compareCols = ['cv','coag','hep','nerv','renal','resp']

# define paths to example csv files
goldStandardFilePath = 'exampledata/goldStandardMetric.csv'
computedFilePath = 'exampledata/computedMetric.csv'

# use Pandas to Load the files
goldStandardDf = pd.read_csv(goldStandardFilePath)
computedDf = pd.read_csv(computedFilePath)

# drop rows with null values
goldStandardDf.dropna(inplace=True)
computedDf.dropna(inplace=True)

# set indexes
goldStandardDf.set_index(indexColName, inplace=True)
computedDf.set_index(indexColName, inplace=True)

# Columns - Here we filter both tables down to the columns we wish to compare
# NOTE: Your incoming dataframes must have the columns in compareCols.  If the load process
# loaded them with different names, you will need to add a renaming step prior to this step
goldStandardDf = goldStandardDf.filter(compareCols)
computedDf = computedDf.filter(compareCols)

# final step add your dataframes to a dictionary
# This allows us to keep track of their content with a named key
dfDict = {
    'Gold Standard': goldStandardDf,
    'Computed':computedDf
}

dfAlignedDict = alignIndexes(dfDict)
pprint(dfAlignedDict)

# Step 2 - glue dataframes together
# Glues the computed dataframe on to the right edge of the gold standard dataframe
# adds a level to the column index which acts as a grouper
# this denotes which data was from which source
df_all = pd.concat(dfAlignedDict.values(),
                   keys=dfAlignedDict.keys(),
                   axis='columns')

df_final = df_all.swaplevel(axis='columns')[sorted(set(x[1] for x in df_all.columns))]

# Apply the diff 
df_final.style.apply(styleDifferences, axis=None, groupAssumedCorrect='Gold Standard', color='red')

### Metric Disagreement Summary
Here we can clearly see there was a systemic disagreement on the Cardiovascular score between the expert labelers and the machine.  Often, finding the single difference leads to finding concordance for the entire set of differences!

In [ ]:
'''Record Error'''
'''Define a few constants'''
# The name of the index column in your data
indexColName = 'patient_id'
# Name of the columns that will be present in both original dataframes you wish to compare
compareCols = ['cv','coag','hep','nerv','renal','resp']

# define paths to example csv files
goldStandardFilePath = 'exampledata/goldStandardRecord.csv'
computedFilePath = 'exampledata/computedRecord.csv'

# use Pandas to Load the files
goldStandardDf = pd.read_csv(goldStandardFilePath)
computedDf = pd.read_csv(computedFilePath)

# drop rows with null values
goldStandardDf.dropna(inplace=True)
computedDf.dropna(inplace=True)

# set indexes
goldStandardDf.set_index(indexColName, inplace=True)
computedDf.set_index(indexColName, inplace=True)

# Columns - Here we filter both tables down to the columns we wish to compare
# NOTE: Your incoming dataframes must have the columns in compareCols.  If the load process
# loaded them with different names, you will need to add a renaming step prior to this step
goldStandardDf = goldStandardDf.filter(compareCols)
computedDf = computedDf.filter(compareCols)

# final step add your dataframes to a dictionary
# This allows us to keep track of their content with a named key
dfDict = {
    'Gold Standard': goldStandardDf,
    'Computed':computedDf
}

dfAlignedDict = alignIndexes(dfDict)
pprint(dfAlignedDict)

# Step 2 - glue dataframes together
# Glues the computed dataframe on to the right edge of the gold standard dataframe
# adds a level to the column index which acts as a grouper
# this denotes which data was from which source
df_all = pd.concat(dfAlignedDict.values(),
                   keys=dfAlignedDict.keys(),
                   axis='columns')

df_final = df_all.swaplevel(axis='columns')[sorted(set(x[1] for x in df_all.columns))]

# Apply the diff 
df_final.style.apply(styleDifferences, axis=None, groupAssumedCorrect='Gold Standard', color='red')

### Record Disagreement Summary
Here, it is clearly seen that record number 4 gave the machine algorithm a problem.  Identifying the root cause of this issue typically brings all scores back into agreement.

# Appendix A - Interrater Agreement
The example provided above relies on having a true Gold Standard dataset for comparative derivation.  To establish a ground truth it is useful to obtain input from several expert humans and adjudicate that input.  Common approaches typically involve an odd number of human annotators in which one of the human annotators is named the 'leading expert' and has the final say in the case of disagreements.  Using the same process as above we can visualize disagreements.  It is also helpful to employ an inter-rater agreement metric as part of this process when quantified agreement statistics are necessary.  Included here are examples of both.  The steps are very similar to those above:
1. Load Data
1. Align Indicies and columns
1. Compare!

In [ ]:
human1FilePath = 'exampledata/human1.csv'
human1Df = pd.read_csv(human1FilePath)
human1Df.dropna(inplace=True)
human1Df.set_index(indexColName, inplace=True)

human2FilePath = 'exampledata/human2.csv'
human2Df = pd.read_csv(human2FilePath)
human2Df.dropna(inplace=True)
human2Df.set_index(indexColName, inplace=True)

human3FilePath = 'exampledata/human3.csv'
human3Df = pd.read_csv(human3FilePath)
human3Df.dropna(inplace=True)
human3Df.set_index(indexColName, inplace=True)

dfDict = {
    'Human1': human1Df,
    'Human2': human2Df,
    'Human3': human3Df
}

In [ ]:
dfAlignedDict = alignIndexes(dfDict)

### Interrater Visual Diff
Same styleDifferences() function can compare more than 2 sources of data!

In [ ]:
df_all = pd.concat(dfAlignedDict.values(),
                   keys=dfAlignedDict.keys(),
                   axis='columns')
df_all.head()

In [ ]:
# Step 3 - Interleave columns
# By swapping the column name grouping levels we can arrange each subscores data value with it's sibling source
df_final = df_all.swaplevel(axis='columns')[sorted(set(x[1] for x in df_all.columns))]
df_final.head()

In [ ]:
# Apply the diff 
df_final.style.apply(styleDifferences, axis=None, groupAssumedCorrect='Human1')

### Interrater Statistics
NOTE: This code takes advantage of NLTK's nicely designed AnnotationTask function.  You must install nltk for this example to work:
```py
pipenv install nltk
```

In [ ]:
from nltk.metrics.agreement import AnnotationTask
# using NLTK's agreement function
def createAgreementData(dfDict):
    '''
    Need to create 3 tuples in the form
    (coder, item, value)
    In this case, we get 
    - coder from the key name in our dictionary
    - item by concatenating the index value and column name
    - value by extracting the value from the dataframe (value of key in our dictionary)
    '''
    data = []
    for k,v in dfDict.items():
        for row in v.itertuples():
            idx=getattr(row,'Index')
            for col in compareCols:
                val = getattr(row, col)
                data.append((k,col+str(idx),val))
    return data

In [ ]:
agreementData = createAgreementData(dfAlignedDict)
agreementStats = AnnotationTask(agreementData)
# Krippendorff
print('Krippendorffs Alpha = {}'.format(agreementStats.alpha()))
# Cohen
print('Cohens Kappa = {}'.format(agreementStats.kappa()))

# Appendix B - Alternative Data Source Examples
Examples of loading data from:
1. Oracle
1. Recap

This is real code from our efforts to derive an automated SOFA score.  Please use these as starting points for your projects!  Refer to README for additional python packages that must be installed for obtaining data from these sources.  The README will also point you to RedCap Project schemas and SQL DDL that allows you to create the underlying data sources in your ecosystem.

## Load Data from Oracle Database

In [ ]:
'''
These additional modules are required for loading data From Oracle
Also note that Python relies on the underlying operating system to make database connections
using the ODBC protocol
For each database engine Oracle, mySQL, Postgres, etc.. you wish to connect to there will be some
OS level dependencies to install, typically some db engine specific client libraries
This approach will work for any database you can create a connection to
'''

import sqlalchemy
import cx_Oracle
# used to mask secrets that might be entered into the notebook
import getpass

In [ ]:
# Database connection
# Here we set up a connection to that schema, prompting for password so it is not saved in the notebook!
dbUser = input("Enter database Username: ")
dbPass = getpass.getpass("Enter database Password: ")
dbUrl = input("Enter database URL: ")

dbEngineURL = 'oracle+cx_oracle://{}:{}@{}'.format(dbUser, dbPass, dbUrl)
dbEngine = sqlalchemy.create_engine(dbEngineURL, pool_size=100)

In [ ]:
"""You will need to edit this sql to match the specifics of your data table!"""
computedSQL = """
    select
        PAT_ENC_CSN_ID,
        TSOFA0TO24COAG,
        TSOFA0TO24CV,
        TSOFA0TO24GCS,
        TSOFA0TO24HEPATIC,
        ESOFA0TO24RENAL,
        TSOFA0TO24RESP
    from
        pydata_tsofa"""

# dictionary used to map sql column names to conformed column names needed for comparison
computedSQLRenameDict = {
    'tsofa0to24coag':'coag',
    'tsofa0to24cv':'cv',
    'tsofa0to24gcs':'cns',
    'tsofa0to24hepatic':'hepatic',
    'esofa0to24renal':'renal',
    'tsofa0to24resp':'resp'
}

In [ ]:
'''
This cell:
1. Reads data from the database using the SQL query defined above
2. Renames the columns to the standard names used in the comparison
3. Sets the index
4. Sorts the index
5. De-dupes the data.  Should not be necessary.  If you notice a change here, review your data source!
'''
computedDf = pd.read_sql_query(computedSQL, dbEngine)
computedDf.rename(columns=computedSQLRenameDict, inplace=True)
computedDf.set_index(['pat_enc_csn_id'], inplace=True)
computedDf.sort_index(inplace=True)

# de-dupe, if necessary
print('Before de-duping the index is unique: {}'.format(computedDf.index.is_unique))
computedDf = (computedDf.reset_index()
                    .drop_duplicates()
                    .set_index('pat_enc_csn_id'))
print('After de-duping the index is unique: {}'.format(computedDf.index.is_unique))

computedDf.info()

## Load Data From Redcap
In our original use case this is where a team of 3 physicians has entered the manually abstracted SOFA scores.  Using the RedCap API, the scores were extracted and converted into a dataframe before feeding into the rest of the data comparison workflow

In [ ]:
# modules used to obtain data from Redcap
import pycurl
from io import StringIO
from io import BytesIO
from urllib.parse import urlencode
import json
# used to mask secrets that might be entered into the notebook
import getpass

In [ ]:
redCapApiToken = getpass.getpass("Please enter your RedCap API Token:")

In [ ]:
'''
This cell:
1. Reads data from the RedCap using the API
2. Renames the columns to the standard names used in the comparison
3. Sets the index
4. Sorts the index
byteBuf = BytesIO()
'''

# Set up post data
data = {
    'token': redCapApiToken,
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'events[0]': 'cr1_arm_1',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
postfields = urlencode(data)

# create CURL object to perform the POST request
ch = pycurl.Curl()
ch.setopt(ch.URL, 'https://redcap.ucdmc.ucdavis.edu/redcap/api/')    
ch.setopt(ch.POSTFIELDS, postfields)
ch.setopt(ch.WRITEDATA, byteBuf)
ch.perform()
ch.close()

# # DECODE the byte buffer return
body = byteBuf.getvalue()
bodyStr = body.decode('utf-8')
bodyJSON = json.loads(bodyStr)
goldstandardDf = pd.DataFrame(bodyJSON)
# turn pat_enc_csn colum to integer column
goldstandardDf['pat_enc_csn_id'] = goldstandardDf['pat_enc_csn_id'].astype('int64')
# Set csn as index
goldstandardDf.set_index('pat_enc_csn_id', inplace=True)
goldstandardDf.sort_index(inplace=True)
# map column names to standards for comparison
renameDict = {'tsofa0to24cns':'cns',
'tsofa0to24coag':'coag',
'tsofa0to24cv':'cv',
'tsofa0to24hepatic':'hepatic',
'tsofa0to24renal':'renal',
'tsofa0to24resp':'resp'}

goldstandardDf = goldstandardDf.rename(columns=renameDict)
# make sofa score cols floats!
numCols = ['coag','cv','cns','hepatic','renal','resp']
goldstandardDf[numCols] = goldstandardDf[numCols].apply(pd.to_numeric, errors='coerce', axis=1)

goldstandardDf.info()